In [12]:
# Convert PDF to text

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/velissarioschristodoulou/Documents/Github Repos/ModalityFinance/modalitybackend/media/FY23-Q1-Combined-NIKE-Press-Release-Schedules-FINAL_bFLU6ao.pdf") 
pages = loader.load_and_split()

In [13]:
text = ""
for page in pages:
    text += page.page_content
print(text)

Investor Contact: Media Contact:
Paul Trussell KeJuan Wilkins
investor.relations@nike.com media.relations@nike.com
NIKE, INC. REPORTS FISCAL 2023  FIRST QUARTER  RESULTS
BEAVERTON, Ore., Sept. 29, 2022  — NIKE, Inc. (NYSE:NKE) today reported fiscal 2023  financial 
results for its first quarter  ended August 31, 2022 . 
•First quarter  reported revenues were $12.7 billion , up 4 percent  compared to the prior year  and 
up 10 percent  on a currency-neutral basis*
•NIKE Direct sales were $5.1 billion , up 8 percent  on a reported basis and up 14 percent  on a 
currency-neutral basis
•NIKE Brand Digital sales increased 16 percent  on a reported basis, or 23 percent on a 
currency-neutral basis, led by 46 percent growth in EMEA
•Gross margin decreased 220 basis points to 44.3 percent
•Diluted earnings per share for the quarter was $0.93 , down 20 percent
"Our strong start to FY23  highlights the depth and breadth of NIKE’s global portfolio, as we 
continue to manage through volatility,” s

In [21]:
# Create a prompt template for parsing information

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate 

template_string_metrics = """ {report} \n
From the above report, extract the following information in a JSON format: \n
* Company name \n
* Date of report \n
* Headquarters \n
* Industry \n 
* Revenue \n 
* Cost of sales \n
* Gross profit \n 
* Gross margin \n
* Net income
* Cash and equivalents \n 

Then compute the EBITDA and also give me that in the JSON output. 

Then explain how you computed the EBITDA
"""

template_string_tables = """ {report} \n
From the above report, extract all the tables and their title. Return them as JSON ready to be saved in a SQL database I have. 
"""

prompt_template_metrics = ChatPromptTemplate.from_template(template=template_string_metrics)
prompt_template_metrics = ChatPromptTemplate.from_template(template=template_string_tables)

In [22]:
chat = ChatOpenAI(temperature=0.7)

input = prompt_template_metrics.format_messages(report=text)
input_tables = prompt_template_metrics.format_messages(report=text)

In [23]:
# output = chat(input)
# print("hooray")
output_tables = chat(input_tables)

In [24]:
# print("First")
# print(output.content)
print("Tables")
print(output_tables.content)

Tables
{
"tables": [
{"title": "NIKE, Inc. CONSOLIDATED STATEMENTS OF INCOME",
"data": [
{"8/31/2022": "$12,687", "8/31/2021": "$12,248", "Change": "4%"},
{"Gross profit": "$5,615", "Gross margin": "44.3%"},
{"Demand creation expense": "$943", "Operating overhead expense": "$2,977", "Total selling and administrative expense": "$3,920"},
{"Interest expense (income), net": "$13", "Income before income taxes": "$1,828", "Income tax expense": "$360", "Effective tax rate": "19.7%"},
{"NET INCOME": "$1,468", "Earnings per common share": {"Basic": "$0.94", "Diluted": "$0.93"}}
]},
{"title": "NIKE, Inc. CONSOLIDATED BALANCE SHEETS",
"data": [
{"August 31, 2022": "$41,088", "August 31, 2021": "$37,917", "% Change": "8%"},
{"Total current assets": "$28,877", "Total current liabilities": "$10,919"},
{"Total long-term debt": "$8,922", "Total shareholders' equity": "$15,822"}
]},
{"title": "NIKE, Inc. DIVISIONAL REVENUES",
"data": [
{"8/31/2022": "$12,687", "8/31/2021": "$12,248", "Change": "4%"},


In [ ]:
print(output_tables.content)

In [13]:
# Convert json object string to a python dictionary
import json

out = json.loads(output.content)

In [29]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate 

# Convert PDF to text
def convert_pdf_to_text(pdf_file):
    """ pdf_file is the filepath """
    loader = PyPDFLoader(pdf_file) 
    pages = loader.load_and_split()
    text = ""
    for page in pages:
        text += page.page_content
    # print(text)
    return text

# Extract income statement and store in database
def extract_income_statement(pdf_file):
    text = convert_pdf_to_text(pdf_file)

    # Create a prompt template for parsing income statement 
    template_string_income_statement = """{report} \n
    From the above report, extract the income statement. Return it as JSON ready to be saved in a SQL database. 
    The format should be like '{{"Revenue": {{"Year 18": 500, "Year 19": 1000}}, "COGS": {{...}}}}'.
    """
    prompt_template_income_statement = ChatPromptTemplate.from_template(template=template_string_income_statement)

    chat = ChatOpenAI(model_name="gpt-4o", temperature=0.7)
    input_tables = prompt_template_income_statement.format_messages(report=text)
    output_tables = chat(input_tables)
    
    return output_tables.content

In [30]:
pdf_file = "/Users/velissarioschristodoulou/Documents/Github Repos/ModalityFinance/modalitybackend/media/FY23-Q1-Combined-NIKE-Press-Release-Schedules-FINAL_bFLU6ao.pdf"

output = extract_income_statement(pdf_file)
print(output)

```json
{
    "Revenue": {
        "8/31/2022": 12687,
        "8/31/2021": 12248
    },
    "Cost of Sales": {
        "8/31/2022": 7072,
        "8/31/2021": 6552
    },
    "Gross Profit": {
        "8/31/2022": 5615,
        "8/31/2021": 5696
    },
    "Gross Margin": {
        "8/31/2022": 44.3,
        "8/31/2021": 46.5
    },
    "Demand Creation Expense": {
        "8/31/2022": 943,
        "8/31/2021": 918
    },
    "Operating Overhead Expense": {
        "8/31/2022": 2977,
        "8/31/2021": 2654
    },
    "Total Selling and Administrative Expense": {
        "8/31/2022": 3920,
        "8/31/2021": 3572
    },
    "Percentage of Revenues": {
        "8/31/2022": 30.9,
        "8/31/2021": 29.2
    },
    "Interest Expense (Income), Net": {
        "8/31/2022": 13,
        "8/31/2021": 57
    },
    "Other (Income) Expense, Net": {
        "8/31/2022": -146,
        "8/31/2021": -39
    },
    "Income Before Income Taxes": {
        "8/31/2022": 1828,
        "8/31/2021":

In [35]:
template_string_map = """ Create a mapping between the names of the fields of the database schema and the names of the income statement fields. Both are provided below \n
JSON: {json_input} \n

Database Schema: 

class IncomeStatement(models.Model):
    statement_id = models.AutoField(primary_key=True)  
    company = models.ForeignKey(Company, on_delete=models.CASCADE)
    fiscal_year = models.IntegerField()
    fiscal_quarter = models.IntegerField(null=True, blank=True)
    reporting_currency = models.CharField(max_length=10)

    total_revenue = models.DecimalField(max_digits=20, decimal_places=2)
    cost_of_goods_sold = models.DecimalField(max_digits=20, decimal_places=2)
    gross_profit = models.DecimalField(max_digits=20, decimal_places=2)

    research_and_development_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    selling_general_administrative_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    depreciation_and_amortization = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    other_operating_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    total_operating_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    operating_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

    interest_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    interest_expense = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    other_non_operating_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    total_non_operating_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    total_non_operating_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    income_before_tax = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

    income_tax_expense = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    net_income_from_continuing_operations = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

    income_from_discontinued_operations = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
    net_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

    earnings_per_share_basic = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)
    earnings_per_share_diluted = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)

    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True) 
    
    Return the output as json so that I can then read it as a dictionary mapping names in my python code. \n
    If a field from the database schema cannot be mapped, put it as "Missing" \n
    For the fields of the income statement that cannot be mapped, return them as a list afterwards """

prompt_template_map = ChatPromptTemplate.from_template(template=template_string_map)

In [36]:
from langchain.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate 

chat = ChatOpenAI(model_name="gpt-4o", temperature=0.7)
input_map = prompt_template_map.format_messages(json_input=output)
output_map = chat(input_map)

print(output_map.content)

Sure, here's the mapping between the fields of the database schema and the income statement fields, along with the list of unmapped fields from the income statement:

```json
{
    "mapping": {
        "total_revenue": "Revenue",
        "cost_of_goods_sold": "Cost of Sales",
        "gross_profit": "Gross Profit",
        "research_and_development_expenses": "Demand Creation Expense",
        "selling_general_administrative_expenses": "Total Selling and Administrative Expense",
        "depreciation_and_amortization": "Missing",
        "other_operating_expenses": "Operating Overhead Expense",
        "total_operating_expenses": "Missing",
        "operating_income": "Missing",
        "interest_income": "Missing",
        "interest_expense": "Interest Expense (Income), Net",
        "other_non_operating_income": "Other (Income) Expense, Net",
        "total_non_operating_income": "Missing",
        "total_non_operating_expenses": "Missing",
        "income_before_tax": "Income Before

In [59]:
import os
import django
import json
import re
from pprint import pprint

# Set up the Django environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "modalitybackend.settings")
django.setup()
print("Setup succesful!")

# Now you can import your models
from myapp.models import IncomeStatement

# Create a new record
new_record = IncomeStatement(fiscal_year='2022')

text = output_map.content
json_match = re.search(r'```json(.*?)```', text, re.DOTALL)
json_str = json_match.group(1).strip()
new_dict = json.loads(json_str)
# print(new_dict)
mapping = new_dict['mapping']
# print(mapping)

# Convert income statement extracted to python dictionary
json_2_match = re.search(r'```json(.*?)```', output, re.DOTALL)
json_2_str = json_2_match.group(1).strip()
income_statement = json.loads(json_2_str)
print("Income Statement: ",income_statement)

for field in mapping:
    if mapping[field] != 'Missing':
        value = income_statement[mapping[field]]["8/31/2022"]
        setattr(new_record, field, value)

print(new_record.total_revenue)
print(new_record.cost_of_goods_sold)

# Convert the object to a dictionary and pretty-print it
pprint(new_record.__dict__)

# print("Record added successfully!")

# class IncomeStatement(models.Model):
#     statement_id = models.AutoField(primary_key=True)  
#     company = models.ForeignKey(Company, on_delete=models.CASCADE)
#     fiscal_year = models.IntegerField()
#     fiscal_quarter = models.IntegerField(null=True, blank=True)
#     reporting_currency = models.CharField(max_length=10)

#     total_revenue = models.DecimalField(max_digits=20, decimal_places=2)
#     cost_of_goods_sold = models.DecimalField(max_digits=20, decimal_places=2)
#     gross_profit = models.DecimalField(max_digits=20, decimal_places=2)

#     research_and_development_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     selling_general_administrative_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     depreciation_and_amortization = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     other_operating_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     total_operating_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     operating_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

#     interest_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     interest_expense = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     other_non_operating_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     total_non_operating_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     total_non_operating_expenses = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     income_before_tax = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

#     income_tax_expense = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     net_income_from_continuing_operations = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

#     income_from_discontinued_operations = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)
#     net_income = models.DecimalField(max_digits=20, decimal_places=2, null=True, blank=True)

#     earnings_per_share_basic = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)
#     earnings_per_share_diluted = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)

#     created_at = models.DateTimeField(auto_now_add=True)
#     updated_at = models.DateTimeField(auto_now=True)

#     class Meta:
#         indexes = [
#             models.Index(fields=['company', 'fiscal_year']),
#             models.Index(fields=['fiscal_year']),
#         ]


Setup succesful!
Income Statement:  {'Revenue': {'8/31/2022': 12687, '8/31/2021': 12248}, 'Cost of Sales': {'8/31/2022': 7072, '8/31/2021': 6552}, 'Gross Profit': {'8/31/2022': 5615, '8/31/2021': 5696}, 'Gross Margin': {'8/31/2022': 44.3, '8/31/2021': 46.5}, 'Demand Creation Expense': {'8/31/2022': 943, '8/31/2021': 918}, 'Operating Overhead Expense': {'8/31/2022': 2977, '8/31/2021': 2654}, 'Total Selling and Administrative Expense': {'8/31/2022': 3920, '8/31/2021': 3572}, 'Percentage of Revenues': {'8/31/2022': 30.9, '8/31/2021': 29.2}, 'Interest Expense (Income), Net': {'8/31/2022': 13, '8/31/2021': 57}, 'Other (Income) Expense, Net': {'8/31/2022': -146, '8/31/2021': -39}, 'Income Before Income Taxes': {'8/31/2022': 1828, '8/31/2021': 2106}, 'Income Tax Expense': {'8/31/2022': 360, '8/31/2021': 232}, 'Effective Tax Rate': {'8/31/2022': 19.7, '8/31/2021': 11.0}, 'Net Income': {'8/31/2022': 1468, '8/31/2021': 1874}, 'Earnings per Common Share - Basic': {'8/31/2022': 0.94, '8/31/2021': 